In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import pathlib
main_path= pathlib.Path('..')
print (main_path.resolve())
sys.path.insert(0, str(main_path.resolve()))

In [ ]:
import logging
logger= logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
import IPython.core.display as display


In [ ]:
from deepface import DeepFace
from stash_ai.config import config, load_config
from stash_ai.db import get_session
from stash_ai.model import StashBox, Performer, PerformerStashBoxImage, Scene, Img, ImgFile, ImageAnalysis, DeepfaceFace
from utils.performer import get_performer_stash_image, load_performer, get_unknown_performer_image
from utils.scene import load_scene, create_or_update_scene_from_stash, extract_scene_images, decord_scene
from utils.image import image_analysis, get_annotated_image_analysis_path, get_face_image_path, get_face_phash, hashes_are_similar
from datetime import timedelta
import random
from PIL import Image
from tqdm import tqdm 
from typing import List
import pandas as pd

In [ ]:
import stash_ai.config as stash_ai_config

stash_ai_config.config_file= main_path.joinpath('config.json')
print(stash_ai_config.config_file.resolve())
load_config()
session= get_session()


In [ ]:
display.JSON(config.stash_interface.find_performers())

In [ ]:
imgFile: ImgFile= session.get(ImgFile, 14270)

#image_analysis: ImageAnalysis= session.get(ImageAnalysis, 1196)
#print(image_analysis)
print(imgFile)

In [ ]:
analysis= image_analysis(imgFile, "mtcnn", 30, session, True)

In [ ]:
for face in analysis.faces:
    print(face)
    for other_face in analysis.faces:
        if other_face.id == face.id:
            continue
        print(face.overlap(other_face))



In [ ]:
im= Image.open(get_annotated_image_analysis_path(analysis, 0.7))
display.display_png(im)

In [ ]:
#ident_faces detector ssd expand 30 min_confidence face 0.9 hash tolerance 15 
state_scene_stash= {'scene_id': 1392}
radio_deepface_ident_detector ='ssd'
number_deepface_ident_expand = 30
number_deepface_ident_face_conf = 0.9
number_ident_hash_tolerance = 15


In [ ]:
dataset_dir= config.data_dir.joinpath('dataset/performers')
columns=["ImgFile Id", "Group", "Confidence", "Face", "Match Face", "Performer Name", "threshold", "distance", "Age", "Gender", "Gender conf.", "Race", "Race conf."]    
data= []
index= []

In [ ]:
data= []
index= []
with get_session() as session:
    scene: Scene= load_scene(state_scene_stash.get('scene_id'), session)
    if scene is None:
        raise gr.Error(f"Scene {state_scene_stash.get('scene_id')} not found in DB!")
    faces: List[DeepfaceFace]= []
    faces_hashes= []
    img: Img            
    for img in scene.images:
        logger.debug(f"detect_and_extract_faces Detection on {img}")
        if not img.original_file_exists():
            logger.warning(f"detect_and_extract_faces File not on disk : {img}")
            continue
        analysis: ImageAnalysis= image_analysis(img.original_file(), radio_deepface_ident_detector, number_deepface_ident_expand, session)

        face: DeepfaceFace
        for face in analysis.faces:
            hash= get_face_phash(face)
            if face.confidence < number_deepface_ident_face_conf:
                continue
            for oh in faces_hashes:
                if hashes_are_similar(hash, oh, number_ident_hash_tolerance):
                    break
            else:
                faces.append(face)
            faces_hashes.append(hash)
        if len(faces) == 2:
            break
    
    face: DeepfaceFace
    for face in faces:
        result= DeepFace.find(get_face_image_path(face), db_path=dataset_dir.resolve(), detector_backend=radio_deepface_ident_detector, expand_percentage=number_deepface_ident_expand)
        df= result[0]
        face_img= get_face_image_path(face)
        for i in range(0, len(df)):
            #print(f"{i}: {df['identity'][i]} {df['threshold'][i]} {df['distance'][i]}")
            match_face_id= int(pathlib.Path(df['identity'][i]).stem)
            match_face= session.get(DeepfaceFace, match_face_id)
            if match_face is not None and match_face.performer is not None:
                name= match_face.performer.name
            else:
                name= f"unknown {match_face_id}"
            index.append(f"{face.id}_{i}")
            data.append((face.image_analysis.image_file.id, 
                         face.group, 
                         face.confidence, 
                         f"<img style='max-height: 75px;' src='/gradio_api/file={face_img}'/>", 
                         f"<img style='max-height: 75px;' src='/gradio_api/file={df['identity'][i]}'/>",
                         name,
                         df['threshold'][i],
                         df['distance'][i],
                         face.age, 
                         face.gender, 
                         face.gender_confidence, 
                         face.race, 
                         face.race_confidence
                         ))


In [ ]:
df

In [ ]:
for i in range(0, len(df)):
    print(f"{i}: {df['identity'][i]} {df['threshold'][i]} {df['distance'][i]}")


In [ ]:
 display.HTML(pd.DataFrame(data=data, index=index, columns=columns).to_html(escape=False))

In [ ]:
p1='https://stash-ai-dev.lan.gatas.ca/gradio_api/file=/opt/stash/stash-ai-helper/local_assets/image_analysis/201_0.9.JPEG'
p2='https://stash-ai-dev.lan.gatas.ca/gradio_api/file=/opt/stash/stash-ai-helper/local_assets/image_analysis/201/144.JPEG'
display.HTML("""
<p>p1 <img style='max-height: 100px;' src='{p1}' /> p2 <img style='max-height: 100px;' src='{p2}' /> </p>
""")